<a href="https://colab.research.google.com/github/fredlarochelle/tweety/blob/main/notebooks/0.1_fl_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import re
import random
import string
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
def get_proxy():
  # https://proxylist.geonode.com/api/organdasn?limit=200&page=2
  response_proxy = requests.get("https://free-proxy-list.net/")

  soup = bs(response_proxy.content, "html.parser")

  tds = soup.find("tbody").find_all("tr")

  proxies = []

  for io in tds:
    proxies.append(f"{io.find_all('td')[0].text}:{io.find_all('td')[1].text}")

  return proxies

In [3]:
def get_headers(typed=None):
  if not typed:
    print("not")
    headers = {
      "authority": "twitter.com",
      "sec-ch-ua": '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
      "x-twitter-client-language": "en",
      "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
      "accept-encoding": "gzip, deflate, br",
      "upgrade-insecure-requests": "1",
      "sec-ch-ua-platform": 'Windows"',
      "sec-ch-ua-mobile": "?0",
      "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36",
    }
  else:
    print("yep")
    headers = {
      'x-csrf-token': ''.join(random.choices(string.ascii_letters + string.digits, k=32)),
      'authorization': "Bearer " + "AAAAAAAAAAAAAAAAAAAAANRILgAAAAAAnNwIzUejRCOuH5E6I8xnZz4puTs%3D1Zv7ttfk8LF81IUq16cHjhLTvJu4FA33AGWWjCpTnA",
      'content-type': "application/json",
      'referer': "https://twitter.com/AmitabhJha3",
      "authority": "twitter.com",
      "sec-ch-ua": '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
      "x-twitter-client-language": "en",
      "upgrade-insecure-requests": "1",
      "sec-ch-ua-platform": 'Windows"',
      "sec-ch-ua-mobile": "?0",
      "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36",
      "x-guest-token": typed
    }
    
  return headers

In [4]:
def get_guest_token():
  try:
    response = requests.get("https://twitter.com/i/trends", headers=get_headers(), proxies=proxy)

    guest_token = re.findall(
      'document\.cookie = decodeURIComponent\("gt=(.*?); Max-Age=10800; Domain=\.twitter\.com; Path=/; Secure"\);',
      response.text)
    
    try:
      return guest_token[0]
    except IndexError:
      raise ValueError("Guest token couldn't be found. Aborting...")
  except:
    raise ValueError("Response didn't work...")

In [5]:
proxy = {"http": random.choice(get_proxy())}
guest_headers = get_headers(get_guest_token())

not
yep


In [6]:
def format_search(response):
  response_json = response.json()

  cursor = []

  for i in response_json['timeline']['instructions'][0]['addEntries']['entries']:
    if 'operation' in i['content']:
      if i['content']['operation']['cursor']['cursorType'] == "Bottom":
        cursor.append(i['content']['operation']['cursor']['value'])

  if len(cursor) == 0:
    for i in response_json['timeline']['instructions']:
      for key in i.keys():
        if key == "replaceEntry":
          if i['replaceEntry']['entry']['content']['operation']['cursor']['cursorType'] == "Bottom":
            cursor.append(i['replaceEntry']['entry']['content']['operation']['cursor']['value'])
            break

  data = []

  for element in response_json['globalObjects']['tweets']:
    data.append([response_json['globalObjects']['tweets'][element]["created_at"], response_json['globalObjects']['tweets'][element]["full_text"], response_json['globalObjects']['tweets'][element]["id"], response_json['globalObjects']['tweets'][element]["user_id"]])

  return data, cursor

In [7]:
search_url = "https://twitter.com/i/api/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweet=true&q={}&count=20&query_source=typeahead_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel%2CvoiceInfo"

def search(query, pages=1):
  url = f"{search_url}&tweet_search_mode=live"
  r = requests.get(url.format(query), headers=guest_headers, proxies=proxy)

  tweets_data, cursor = format_search(r)

  for i in range(pages):
    url = f"{search_url}&tweet_search_mode=live&cursor={cursor[0]}"
    r = requests.get(url.format(query), headers=guest_headers, proxies=proxy)
    tweets_data_temp, cursor = format_search(r)

    tweets_data.extend(tweets_data_temp)

  return pd.DataFrame(tweets_data, columns=["created_at", "full_text", "id", "user_id"])

In [8]:
%time test = search('canada', pages=20)

CPU times: user 357 ms, sys: 14.2 ms, total: 371 ms
Wall time: 7.13 s


In [9]:
test

,created_at,full_text,id,user_id
0,Thu Dec 23 06:12:31 +0000 2021,@aracelibs Nosotros viajamos a Canadá a princi...,1473899300090351618,1428159371805040644
1,Thu Dec 23 06:11:47 +0000 2021,"Canada sees record-breaking 14,456 new COVID-1...",1473899115838853123,1430853682824876033
2,Thu Dec 23 06:12:12 +0000 2021,Just read this from Gov't of Canada's website:...,1473899218507075587,1403458233805721601
3,Thu Dec 23 06:11:48 +0000 2021,It's 11:11 in…\n🇺🇸 United States (MST)\n🇨🇦 Can...,1473899118544101380,1399327423020208133
4,Thu Dec 23 06:12:28 +0000 2021,@DickZoutman I don't understand why @JustinTru...,1473899286362570759,1366220072860196865
...,...,...,...,...
426,Thu Dec 23 05:54:50 +0000 2021,"點開網站 ""Available in Canada"" ...",1473894849867055105,92580306
427,Thu Dec 23 05:54:54 +0000 2021,@CTVToronto @erinotoole @NewYorkStateAG @Charl...,1473894865029185545,1363008538784727041
428,Wed Dec 22 23:01:15 +0000 2021,@WhiteCrkLumber https://t.co/an9URMpfyb,1473790766958800896,4749623750
429,Thu Dec 23 05:54:54 +0000 2021,@Barspin1 @BurnerTeddy @thepurpletart @wballs3...,1473894865255686144,1425964864594452488
